In [17]:
#Download the model
!git clone https://github.com/philshiu/Drosophila_brain_model.git
!pip install brian2

Cloning into 'Drosophila_brain_model'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 120 (delta 39), reused 63 (delta 13), pack-reused 5
Receiving objects: 100% (120/120), 89.23 MiB | 51.22 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [18]:
cd Drosophila_brain_model

/content/Drosophila_brain_model/Drosophila_brain_model/Drosophila_brain_model


In [19]:
from model import run_exp
from model import default_params as params
import utils as utl
from brian2 import Hz

config = {
    'path_res'  : './results/example',                              # directory to store results
    'path_comp' : './2023_03_23_completeness_630_final.csv',        # csv of the complete list of Flywire neurons
    'path_con'  : './2023_03_23_connectivity_630_final.parquet',    # connectivity data
    'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
}

# Introduction
## Underlying connectivity data
The connectivity of the fly brain is stored in the folowing files:
- neurons present: `config['path_comp']`
- connectivity between neurons: `config['path_con]`

## Model parameters
The equation and constants for the leaky integrate and fire model are defined
in the dictionary `default_params` in the beginning of the file `model.py`:

```
default_params = {
    # trials
    't_run'     : 1000 * ms,              # duration of trial
    'n_run'     : 30,                     # number of runs

    'v_0'       : -52 * mV,               # resting potential
    'v_rst'     : -52 * mV,               # reset potential after spike
    [...]
```
We can also change v
alues and pass the modified dictionary to the model (see Experiment 1).

## Addressing neurons
Here, we want to stimulate some sugar-sensing neurons in the right hemisphere.
The neurons of interest are defined via their flywire IDs:

In [ ]:
neu_sugar = [
    720575940624963786,
    720575940630233916,
    720575940637568838,
    720575940638202345,
    720575940617000768,
    720575940630797113,
    720575940632889389,
    720575940621754367,
    720575940621502051,
    720575940640649691,
    720575940639332736,
    720575940616885538,
    720575940639198653,
    720575940620900446,
    720575940617937543,
    720575940632425919,
    720575940633143833,
    720575940612670570,
    720575940628853239,
    720575940629176663,
    720575940611875570,
]

For an easier identification, we define also a mapping from the flywire IDs to custom
names. The above neurons are calles `sugar_1`, `sugar_2` etc:

In [ ]:
flyid2name = { f: 'sugar_{}'.format(i+1) for i, f in enumerate(neu_sugar) }
flyid2name

{720575940624963786: 'sugar_1',
 720575940630233916: 'sugar_2',
 720575940637568838: 'sugar_3',
 720575940638202345: 'sugar_4',
 720575940617000768: 'sugar_5',
 720575940630797113: 'sugar_6',
 720575940632889389: 'sugar_7',
 720575940621754367: 'sugar_8',
 720575940621502051: 'sugar_9',
 720575940640649691: 'sugar_10',
 720575940639332736: 'sugar_11',
 720575940616885538: 'sugar_12',
 720575940639198653: 'sugar_13',
 720575940620900446: 'sugar_14',
 720575940617937543: 'sugar_15',
 720575940632425919: 'sugar_16',
 720575940633143833: 'sugar_17',
 720575940612670570: 'sugar_18',
 720575940628853239: 'sugar_19',
 720575940629176663: 'sugar_20',
 720575940611875570: 'sugar_21'}

# Running simulations
## Activating a set of neurons
To run a simulation exciting these nerons we have to call `run_exp` supplying the following:
- unique name for the simulation: `exp_name`
- a list of neurons we want to stimulate: `neu_sugar`
- the connectivity data: `config['path_comp']` and `config['path_con]`
- path to store the output: `config['path_res']`
- number of CPU cores use: `config['n_procs]`

Note that running this on Google Colab can take roughly 20 minutes; it is substantially faster on a local install, depending on the number of CPU cores. By default, the neurons are excited at 200 Hz.

In [ ]:
# activate sugar sensing neurons
run_exp(exp_name='sugarR', neu_exc=neu_sugar, **config)

>>> Experiment:     sugarR_2
    Output file:    results/example/sugarR_2.parquet
    Excited neurons: 21
    Elapsed time:   1242 s


The `.parquet` file created during a simulation contains all spikes events of all neurons in the model.
We load the data again from disk by passing a list of result files to the `utl.load_exps` function.

We can see from the size of the dataframe
that more than 400 000 spikes were generated by activating the sugar neurons (30 trials, 1 s each).

In [ ]:
# load data from disk
df_spike = utl.load_exps([ './results/example/sugarR.parquet' ])
df_spike

,t,trial,flywire_id,exp_name
0,0.8131,0,720575940605420902,sugarR_2
1,0.1890,0,720575940605513649,sugarR_2
2,0.2480,0,720575940605513649,sugarR_2
3,0.3992,0,720575940605513649,sugarR_2
4,0.4852,0,720575940605513649,sugarR_2
...,...,...,...,...
406291,0.9101,29,720575940660229505,sugarR_2
406292,0.9247,29,720575940660229505,sugarR_2
406293,0.9500,29,720575940660229505,sugarR_2
406294,0.9687,29,720575940660229505,sugarR_2


The spike times can be converted to spike rates [Hz] via `utl.get_rate`, which requires the duration of each trial.
`utl.get_rate` returns `pandas.DataFrame` objects:
1. spike rate for each neuron (rows) in each experiment (column): `df_rate`
2. standard deviation of rate across trials: `df_rate_std`

For convenience, we can optionally pass the `flyid2name` dictionary to `utl.get_rate` in order to convert flywire IDs into
meaningful names.

We can see that only about 400 neurons show activity during the simulations.

In [ ]:
# calculate spike rate and standard deviation
df_rate, df_rate_std = utl.get_rate(df_spike, duration=params['t_run'], flyid2name=flyid2name)
# sort by spike rate
df_rate.sort_values('sugarR', ascending=False)

exp_name,name,sugarR_2
flyid,,
720575940612670570,sugar_18,150.766667
720575940639332736,sugar_11,150.700000
720575940630797113,sugar_6,150.500000
720575940633143833,sugar_17,150.400000
720575940629176663,sugar_20,150.166667
...,...,...
720575940625779075,,1.000000
720575940625561406,,1.000000
720575940624986919,,1.000000


##Save the data
To save the data, connect to your google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df_rate.sort_values('sugarR', ascending=False).to_csv('/content/drive/MyDrive/example_data.csv')

## Change stimulation frequency

We want to change the frequency of the stimulation of the sugar neurons.
To do so we modify the value for `r_poi` in the `default_params` dictionary and pass the altered dictionary to the `run_exp` function.

Note: Since physical quantities in `brian2` have to have the correct unit, we also need the `brian2.Hz` object
to define a frequency.

In [30]:
# run with different frequency
params['r_poi'] = 100 * Hz

run_exp(exp_name='sugarR_100Hz', neu_exc=neu_sugar, params=params, **config)

>>> Skipping experiment sugarR_100Hz because results/example/sugarR_100Hz.parquet exists and force_overwrite = False


We load the results via the `utl.load_exps` function and convert the spike events to rates with `utl.get_rate`

In [31]:
ps = [
    './results/example/sugarR.parquet',
    './results/example/sugarR_100Hz.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, duration=params['t_run'], flyid2name=flyid2name)
df_rate.sort_values('sugarR_100Hz', ascending=False, inplace=True)
df_rate

exp_name,name,sugarR,sugarR_100Hz
flyid,,,
720575940622695448,,156.766667,114.266667
720575940621754367,sugar_8,194.100000,102.566667
720575940617937543,sugar_15,199.400000,101.733333
720575940629888530,,146.500000,101.700000
720575940629176663,sugar_20,191.866667,101.266667
...,...,...,...
720575940639813365,,1.000000,NaN
720575940643551392,,1.222222,NaN
720575940644666148,,1.619048,NaN


## Silencing neurons
We want to silence the most active neurons individually to see how that changes the activity patterns.
We do so by passing the neuron IDs we want to silence as a list `run_exp` via the `neu_slnc` argument.
In the following example, we are silencing a single neuron `[ i ]` while exciting the sugar neurons `neu_sugar`.
We can then investigate how silencing of each individual neuron affects the firing rate of a given neuron, say, MN9.

In [34]:
#First, let's check on the MN9 firing rate when no neurons are silenced.
id_mn9 = 720575940660219265 #id for MN9
df_rate.loc[id_mn9, "sugarR_100Hz"]

67.03333333333333

In [41]:
# IDs of 3 most active neurons. These neurons are all sugar-sensing neurons.
ids = df_rate.sort_values('sugarR_100Hz', ascending=False).index[:3]
for i in ids:
    run_exp(exp_name='sugarR-{}'.format(i), neu_exc=neu_sugar, neu_slnc=[ i ], params=params, **config)

>>> Skipping experiment sugarR-720575940622695448 because results/example/sugarR-720575940622695448.parquet exists and force_overwrite = False
>>> Skipping experiment sugarR-720575940621754367 because results/example/sugarR-720575940621754367.parquet exists and force_overwrite = False
>>> Skipping experiment sugarR-720575940617937543 because results/example/sugarR-720575940617937543.parquet exists and force_overwrite = False


In [42]:
# output files
ps = [ './results/example/sugarR-{}.parquet'.format(i) for i in ids ]

# calculate spike rate and sort
df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, duration=params['t_run'])
df_rate.loc[id_mn9, :].sort_values(ascending=True)

exp_name
sugarR-720575940617937543    63.266667
sugarR-720575940621754367    66.900000
sugarR-720575940622695448    71.166667
Name: 720575940660219265, dtype: float64